# Version information

In [ ]:
from datetime import date
print("Running date:", date.today().strftime("%B %d, %Y"))
import pyleecan
print("Pyleecan version:" + pyleecan.__version__)
import SciDataTool
print("SciDataTool version:" + SciDataTool.__version__)

# How to plot results

Pyleecan has generic built-in plot methods which allow to easily plot:

- **simulated** or **experimental** fields (unit conversions supported)

- **slices** (normalizations available)

- **Fourier Transforms**

but also to **compare** on the same graph a list of fields.

This flexibility is provided by the [SciDataTool module](https://github.com/Eomys/SciDataTool), which has been created to **ease the handling of scientific data**, and considerately simplify plot commands. It unifies the extraction of relevant data (e.g. slices), whether they are stored in the time/space or in the frequency domain. The call to Fourier Transform functions is **transparent**, although it still can be parameterized.

This tutorial explains how to use Pyleecan's **plot commands**. It is available on [GitHub](https://github.com/Eomys/pyleecan/tree/master/Tutorials/tuto_Plots.ipynb). For more details on the `SciDataTool` package, please refer to the specific tutorials:

- [How to create Data objects](https://nbviewer.jupyter.org/github/Eomys/SciDataTool/blob/master/Tutorials/tuto1_Create.ipynb)

- [How to extract slices](https://nbviewer.jupyter.org/github/Eomys/SciDataTool/blob/master/Tutorials/tuto2_Slices.ipynb)

- [How to plot data](https://nbviewer.jupyter.org/github/Eomys/SciDataTool/blob/master/Tutorials/tuto3_Plot.ipynb)

To demonstrate the capabilities and the use of the SciDataTool objects, two simulations are launched with FEMM: with imposed currents and in open-circuit. The simulations use periodicity and parallelization to reduce execution time.

In [ ]:
# Import Pyleecan modules

import matplotlib.pyplot as plt
import numpy as np
from os.path import dirname, join
import matplotlib.pyplot as plt
import numpy as np
import pytest
from numpy import exp, linspace, ones, pi, sqrt, zeros
from pyleecan.Classes.DataKeeper import DataKeeper
from pyleecan.Classes.DriveWave import DriveWave
from pyleecan.Classes.EEC_PMSM import EEC_PMSM
from pyleecan.Classes.ForceMT import ForceMT
from pyleecan.Classes.ImportGenMatrixSin import ImportGenMatrixSin
from pyleecan.Classes.ImportGenVectLin import ImportGenVectLin
from pyleecan.Classes.ImportGenVectSin import ImportGenVectSin
from pyleecan.Classes.ImportMatrixVal import ImportMatrixVal
from pyleecan.Classes.InputCurrent import InputCurrent
from pyleecan.Classes.MagFEMM import MagFEMM
from pyleecan.Classes.OPdq import OPdq
from pyleecan.Classes.Output import Output
from pyleecan.Classes.ParamExplorerInterval import ParamExplorerInterval
from pyleecan.Classes.ParamExplorerSet import ParamExplorerSet
from pyleecan.Classes.PostFunction import PostFunction
from pyleecan.Classes.PostPlot import PostPlot
from pyleecan.Classes.Simu1 import Simu1
from pyleecan.Classes.VarLoadCurrent import VarLoadCurrent
from pyleecan.Classes.VarParam import VarParam
from pyleecan.definitions import DATA_DIR
from pyleecan.Functions.load import load
#from Tests import save_validation_path as save_path



In [1]:
from os.path import join
from multiprocessing import cpu_count
import pytest
from numpy import array, ones, pi, zeros, sqrt
from pyleecan.Classes.InputCurrent import InputCurrent
from pyleecan.Classes.MagFEMM import MagFEMM
from pyleecan.Classes.OPdq import OPdq
from pyleecan.Classes.Simu1 import Simu1
from pyleecan.Classes.VarLoadCurrent import VarLoadCurrent
from pyleecan.definitions import DATA_DIR
from pyleecan.Functions.load import load
from SciDataTool.Functions.Plot.plot_2D import plot_2D
from pyleecan.definitions import config_dict


In [14]:
from SciDataTool.GUI.DDataPlotter import DDataPlotter
DDataPlotter.Ui_DDataPlotter.setupUi()

TypeError: setupUi() missing 2 required positional arguments: 'self' and 'DDataPlotter'

In [2]:
"""Validation of a SynRM machine from Syr-e r29 open source software
https://sourceforge.net/projects/syr-e/
Test compute the Torque in FEMM as a function of Phi0
and compare the results with Syr-e r29
"""
# The aim of this validation test is to compute the torque as a function of Phi0
# As (for now) there is no electrical model, we will compute the current for each Phi0 here
SynRM_001 = load(join(DATA_DIR, "Machine", "SynRM_001.json"))
felec = 50  # supply frequency [Hz]
Nt_tot = 3  # Number of time step for each current angle Phi0
Imax = 28.6878 / sqrt(2)  # RMS stator current magnitude [A]
# to have one torque ripple period since torque ripple appears at multiple of 6*felec
p = SynRM_001.stator.get_pole_pair_number()
N0 = 60 * felec / p

Phi0 = (
    array(
        [
            0.0,
            15.0,
            30.0,
            45.0,
            60.0,
            67.5,
            75.0,
            85.0,
            90.0,
            95.0,
            105.0,
            120.0,
            135.0,
            150.0,
            165.0,
            180.0,
        ]
    )
    * pi
    / 180
)
# Expected results
Tem = [
    0.08,
    7.09,
    13.95,
    19.75,
    23.75,
    23.63,
    21.62,
    8.7655,
    -0.21,
    -8.8544,
    -21.316,
    -23.73,
    -20.20,
    -13.99,
    -7.5445,
    0.08,
]

# Definition of the main simulation
simu = Simu1(name="test_FEMM_torque", machine=SynRM_001)
Na_tot = 2016

varload = VarLoadCurrent(is_reuse_femm_file=True)
varload.type_OP_matrix = 0  # Matrix N0, I0, Phi0, Tem_ref

N_simu = Phi0.size
OP_matrix = zeros((N_simu, 4))
OP_matrix[:, 0] = ones(N_simu) * N0
OP_matrix[:, 1] = ones(N_simu) * Imax
OP_matrix[:, 2] = Phi0
OP_matrix[:, 3] = Tem
varload.OP_matrix = OP_matrix
simu.var_simu = varload

simu.input = InputCurrent(
    Is=None,
    Ir=None,  # No winding on the rotor
    OP=OPdq(N0=N0, felec=felec),
    Nt_tot=Nt_tot,
    Nrev=1 / 6,
    Na_tot=Na_tot,
)
# Select first OP as reference
simu.input.set_OP_from_array(OP_matrix, type_OP_matrix=varload.type_OP_matrix)

# Definition of the magnetic simulation (1/2 symmetry)
simu.mag = MagFEMM(
    type_BH_stator=0,
    type_BH_rotor=0,
    is_periodicity_a=True,
    is_periodicity_t=False,
    nb_worker=cpu_count(),
)
simu.force = None
simu.struct = None

Xout = simu.run()

curve_colors = config_dict["PLOT"]["COLOR_DICT"]["CURVE_COLORS"]
plot_2D(
    array([x * 180 / pi for x in Xout.xoutput_dict["Phi0"].result]),
    [Xout.xoutput_dict["Tem_av"].result, Xout.xoutput_dict["Tem_av_ref"].result],
    color_list=curve_colors,
    legend_list=["Pyleecan", "Syr-e r29"],
    xlabel="Current angle [°]",
    ylabel="Electrical torque [N.m]",
    title="Electrical torque vs current angle",
    save_path=join(save_path, "test_FEMM_torque.png"),
    is_show_fig=False,
)




[05:32:07] Starting running simulation test_FEMM_torque (machine=SynRM_001)


TypeError: 'property' object is not callable

In [ ]:
# Prius MTPA
N0_MTPA = [
    500,
    894.736842105263,
    1289.47368421053,
    1684.21052631579,
    2078.94736842105,
    2473.68421052632,
    2868.42105263158,
    3263.15789473684,
    3657.89473684211,
    4052.63157894737,
    4447.36842105263,
    4842.10526315790,
    5236.84210526316,
    5631.57894736842,
    6026.31578947368,
    6421.05263157895,
    6815.78947368421,
    7210.52631578947,
    7605.26315789474,
    8000,
]
Id_MTPA = [
    -135.671342685371,
    -135.671342685371,
    -135.671342685371,
    -155.310621242485,
    -151.803607214429,
    -128.657314629259,
    -123.046092184369,
    -113.927855711423,
    -104.108216432866,
    -99.8997995991984,
    -94.9899799599199,
    -94.9899799599199,
    -92.1843687374750,
    -92.1843687374750,
    -88.6773547094188,
    -87.2745490981964,
    -87.2745490981964,
    -88.6773547094188,
    -84.4689378757515,
    -88.6773547094188,
]
Iq_MTPA = [
    113.226452905812,
    113.226452905812,
    113.226452905812,
    83.6673346693387,
    54.6092184368737,
    45.0901803607214,
    36.0721442885772,
    30.5611222444890,
    28.0561122244489,
    25.5511022044088,
    23.5470941883768,
    21.5430861723447,
    20.0400801603206,
    18.5370741482966,
    17.5350701402806,
    16.5330661322645,
    15.5310621242485,
    14.5290581162325,
    14.0280561122245,
    13.0260521042084,
]



In [ ]:
save_path='D:\KDH\pyleecan_out_figure'
# Main loop parameters
Nt_tot = 96  # Number of time step for each FEMM simulation
nb_worker = 3  # To parallelize FEMM

Nspeed = 7  # Number of speed for the Variable speed linspace

N1 = 4  # Number of parameters for first sensitivity parameter
# N2 = 2  # Number of parameters for second sensitivity parameter

# Reference simulation definition
Toyota_Prius = load(join(DATA_DIR, "Machine", "Toyota_Prius.json"))
simu = Simu1(
    name="test_multi_multi",
    machine=Toyota_Prius,
    path_result=join(save_path, "test_multi_multi"),
    layer_log_warn=2,
)

# Enforced sinusoïdal current (Maximum Torque Per Amp)
simu.input = InputCurrent(
    Is=None,
    Ir=None,  # No winding on the rotor
    OP=OPdq(N0=N0_MTPA[0], Id_ref=Id_MTPA[0], Iq_ref=Iq_MTPA[0]),
    Nt_tot=Nt_tot,
    Na_tot=2048,
)

# Definition of the magnetic simulation
simu.mag = MagFEMM(
    type_BH_stator=0,
    type_BH_rotor=0,
    is_periodicity_a=True,
    is_periodicity_t=True,
    Kgeo_fineness=0.2,
    Kmesh_fineness=0.2,
    nb_worker=nb_worker,
)
simu.force = ForceMT(
    is_periodicity_a=True,
    is_periodicity_t=True,
)


In [ ]:
result=simu.run_single()

In [ ]:

# VarSpeed Definition
varload = VarLoadCurrent(is_reuse_femm_file=True)
varload.type_OP_matrix = 1  # Matrix N0, Id, Iq

OP_matrix = zeros((Nspeed, 3))
OP_matrix[:, 0] = N0_MTPA[:Nspeed]
OP_matrix[:, 1] = Id_MTPA[:Nspeed]
OP_matrix[:, 2] = Iq_MTPA[:Nspeed]
varload.OP_matrix = OP_matrix
varload.datakeeper_list = [
    DataKeeper(
        name="Average Torque",
        unit="N.m",
        symbol="Tem_av",
        keeper="lambda output: output.mag.Tem_av",
        error_keeper="lambda simu: np.nan",
    ),
    DataKeeper(
        name="6f Harmonic",
        unit="N.m^2",
        symbol="6fs",
        keeper="lambda output: output.force.AGSF.components['radial'].get_magnitude_along('freqs->elec_order=6','wavenumber=0')['AGSF_r']",
        error_keeper="lambda simu: np.nan",
    ),
    DataKeeper(
    name="12f Harmonic",
    unit="N.m^2",
    symbol="12fs",
    keeper="lambda output: output.force.AGSF.components['radial'].get_magnitude_along('freqs->elec_order=12','wavenumber=0')['AGSF_r']",
    error_keeper="lambda simu: np.nan",
),
]
varload.is_keep_all_output = False

# Multi-simulation to change machine parameters
multisim = VarParam(
    stop_if_error=True,
    is_reuse_femm_file=False,
)

simu.var_simu = multisim



In [ ]:
# List of ParamExplorer to define multisimulation input values
paramexplorer_list = [
ParamExplorerInterval(
    name="Stator slot opening",
    symbol="W0s",
    unit="m",
    setter="simu.machine.stator.slot.W0",
    getter="simu.machine.stator.slot.W0",
    min_value=0.1 * Toyota_Prius.stator.slot.W0,
    max_value=Toyota_Prius.stator.slot.W0,
    N=N1,
)
]
multisim.paramexplorer_list = paramexplorer_list
multisim.is_keep_all_output = True

In [ ]:
# List of DataKeeper to store results
datakeeper_list = [
    DataKeeper(
        name="Max Variable speed Torque",
        unit="N.m",
        symbol="Max_Tem_av",
        keeper="lambda output: max(output.xoutput_dict['Tem_av'].result)",
        error_keeper="lambda simu: np.nan",
    ),
    DataKeeper(
        name="Max f6",
        unit="N.m^2",
        symbol="max(6fs)",
        keeper="lambda output: max(output.xoutput_dict['6fs'].result)",
        error_keeper="lambda simu: np.nan",
    ),
    DataKeeper(
        name="Max f12",
        unit="N.m^2",
        symbol="max(12fs)",
        keeper="lambda output: max(output.xoutput_dict['12fs'].result)",
        error_keeper="lambda simu: np.nan",
    ),
]

In [ ]:
multisim.datakeeper_list = datakeeper_list
multisim.var_simu = varload  # Setup Multisim of Multi_sim


In [ ]:
# Post-process
#Post1 = PostFunction("plot_save.py")

#simu.postproc_list = [Post1]  # For all simulation save a png
# Plot Max(f6) = f(W0)
Post2 = PostPlot(
    method="plot_multi",
    param_list=["W0s", "max(6fs)"],
    param_dict={
        "save_path": join(save_path, "multi_multi", "Max_6fs.png"),
        "is_show_fig": False,
    },
)
# Plot Max(f12) = f(W0)
Post3 = PostPlot(
    method="plot_multi",
    param_list=["W0s", "max(12fs)"],
    param_dict={
        "save_path": join(save_path, "multi_multi", "Max_12fs.png"),
        "is_show_fig": False,
    },
)
# Generate gif once all the simulation are done
Post4 = PostFunction(join(dirname(__file__), "make_gif.py"))
simu.var_simu.postproc_list = [Post2, Post3, Post4]




In [ ]:
# Execute every simulation
results = simu.run()

Out

The following example demonstrates the syntax to **quickly plot** the torque and airgap flux obtained at the end of the simulation:

In [ ]:
%matplotlib notebook
#------------------------------------------------------
# Plot the torque as a function of time
out.mag.Tem.plot_2D_Data("time")
out.mag.B.plot_2D_Data("angle")
#------------------------------------------------------

The plot command requires the **axis** of the field along which to plot ("*angle*" for instance). Titles and legends are automatically generated. The commands work with **any field computed in Pyleecan** (and stored in a SciDataTool object).

The syntax for the axes is specific to `SciDataTool` objects, and will be detailed in this tutorial.

There are two main plot commands in `SciDataTool`, which can apply to any **DataTime**, **DataFreq** or **VectorField** object from `SciDataTool`:

- `plot_2D_Data`: field as a function of one dimension (time, angle, freqs, wavenumber, phase
- `plot_3D_Data`: field as a function of two dimensions

`SciDataTool` also provides generic plot functions, which can be useful to easily format other plots, or plots of data which is not stored in a `SciDataTool` object:

- `plot_2D`: plot as a function of one dimension (curve, bargraph, barchart, quiver, curve_point, point, barStackResultant)
- `plot_3D`: plot as a function of two dimensions (stem, surf, pcolor, scatter)
- `plot_4D`: plot as a function of three dimensions (scatter)

To use one of these plot functions, simply import it such as:

In [ ]:
from SciDataTool.Functions.Plot.plot_2D import plot_2D

We invite you to visit [SciDataTool repository on GitHub](https://github.com/Eomys/SciDataTool/tree/master/SciDataTool/Functions/Plot) for more details.

Fonts and colors can be automatically defined from the config_dict, using `dict_2D` or `dict_3D`:

In [ ]:
from pyleecan.Functions.Plot import dict_2D, dict_3D

## 1. How to plot only one period/antiperiod
To plot only one period or anti-period:

In [ ]:
#---------------------------------------------------------------
# Plot the torque along the smallest period of time
out.mag.Tem.plot_2D_Data("time[smallestperiod]", **dict_2D)
#---------------------------------------------------------------

## 2. How to plot VectorField objects
For `VectorField` objects, such as `mag.B` or `force.AGSF`, a plot for each component will be created:

In [ ]:
#---------------------------------------------------------------
# Plot all components of flux density
out.mag.B.plot_2D_Data("time", **dict_2D)
#---------------------------------------------------------------

To plot only one or several specific components, one can use the `component_list` parameter:

In [ ]:
#---------------------------------------------------------------
# Plot only radial flux density
out.mag.B.plot_2D_Data("time", component_list=["radial"], **dict_2D)
#---------------------------------------------------------------

## 3. How to plot slices

In the previous plots, the flux density was sliced at `angle=0` (slice by default), but we could want to slice at another angle:

In [ ]:
#---------------------------------------------------------------
# Plot for angle = 90°
out.mag.B.plot_2D_Data("time", "angle=90{°}", component_list=["radial"], **dict_2D)
#---------------------------------------------------------------

We can even superimpose several slices:

In [ ]:
#---------------------------------------------------------------
# Plot for 3 slices of time
out.mag.B.plot_2D_Data("angle", "time[0,20,40]", component_list=["radial"], **dict_2D)
#---------------------------------------------------------------

## 4. How to plot spectra

One of the main interest of `SciDataTool` is to hide the complexity of Fourier Transforms. From a field defined in the time/space domain, you can directly plot its spectrum (note that intervals can be easily specified):

In [ ]:
#---------------------------------------------------------------
# Plot time fft
out.mag.B.plot_2D_Data("freqs", component_list=["radial"], **dict_2D)
# Plot space fft
out.mag.B.plot_2D_Data("wavenumber=[0,200]", component_list=["radial"], **dict_2D)
#---------------------------------------------------------------

The main frequencies/wavenumbers are automatically labelled. This can be deactivated using `is_auto_ticks=False`:

In [ ]:
##---------------------------------------------------------------
# Deactivate automatic ticks
out.mag.B.plot_2D_Data("wavenumber=[0,200]", is_auto_ticks=False, component_list=["radial"], **dict_2D)
#---------------------------------------------------------------

## 5. How to convert or normalize data

Another interesting feature of `pyleecan` is the conversion feature: there are built-in methods to convert units, and normalize fields and axes.

For example, we can plot the fft in electrical orders:

In [ ]:
#---------------------------------------------------------------
# Normalize frequency axis
out.mag.B.plot_2D_Data("freqs->elec_order", component_list=["radial"], **dict_2D)
#---------------------------------------------------------------

In the following example, we will convert the stator winding flux from Webers into Maxwells ($1$ Wb = $10^8$ Mx):

In [ ]:
#---------------------------------------------------------------
# Original plot
out.mag.Phi_wind_stator.plot_2D_Data("time", "phase[]", **dict_2D)
#---------------------------------------------------------------

In [ ]:
#---------------------------------------------------------------
# Conversion into Maxwells
out.mag.Phi_wind_stator.plot_2D_Data("time", "phase[]", unit="Mx", **dict_2D)
#---------------------------------------------------------------

In this last example, we will convert the angle axis into degrees and for the air-gap flux density from Teslas into Gauss ($1$ T= $10^4$ G)

In [ ]:
#---------------------------------------------------------------
# Original plot
out.mag.B.plot_2D_Data("angle", component_list=["radial"], **dict_2D)
#---------------------------------------------------------------

In [ ]:
#---------------------------------------------------------------
# Original plot
out.mag.B.plot_2D_Data("angle{°}", component_list=["radial"], unit="G", **dict_2D)
#---------------------------------------------------------------

## 7) How to compare data

`pyleecan` also allows to easily compare several fields, even if they are defined on different discretizations. To do so, plot commands have a `data_list` parameter.

In the following example, we compare the flux density from the reference simulation, and from the open-circuit one:

In [ ]:
#---------------------------------------------------------------
# compare simu and simu2
out.mag.B.plot_2D_Data(
    "time", component_list=["radial"], data_list=[out2.mag.B], legend_list=["Reference", "Open-circuit"], **dict_2D
)
#---------------------------------------------------------------

We can also compare spectra:

In [ ]:
#---------------------------------------------------------------
# compare simu and simu2
out.mag.B.plot_2D_Data(
    "freqs<1000", component_list=["radial"], data_list=[out2.mag.B], legend_list=["Reference", "Open-circuit"], **dict_2D
)
#---------------------------------------------------------------

## 8) 3D plots

3D plots (for data defined on at least two axes) can also be created using the `plot_3D_Data` command:

In [ ]:
#---------------------------------------------------------------
# 3D surface plot
out.mag.B.plot_3D_Data("time", "angle{°}", component_list=["radial"], **dict_3D)
#---------------------------------------------------------------

To visualize this surface from above, use `is_2D_view`:

In [ ]:
#---------------------------------------------------------------
# 2D surface plot
out.mag.B.plot_3D_Data("time", "angle{°}", component_list=["radial"], is_2D_view=True, **dict_3D)
#---------------------------------------------------------------

The field is automatically reconstructed using the periodicities specified in the simulation. It is always possible to plot a single period:

In [ ]:
#---------------------------------------------------------------
# 2D surface plot over one period
out.mag.B.plot_3D_Data("time[smallestperiod]", "angle[smallestperiod]{°}", component_list=["radial"], is_2D_view=True, **dict_3D)
#---------------------------------------------------------------

2D Fourier Transforms also rely on `plot_3D_Data` command:

In [ ]:
#---------------------------------------------------------------
# 3D stem plot of 2D fft
out.mag.B.plot_3D_Data(
    "freqs->elec_order=[0,10]", "wavenumber->space_order=[-10,10]", N_stem=50, component_list=["radial"], **dict_3D
)
#---------------------------------------------------------------

It can also be viewed in 2D:

In [ ]:
#---------------------------------------------------------------
# 2D plot of 2D fft
out.mag.B.plot_3D_Data(
    "freqs->elec_order=[0,10]",
    "wavenumber->space_order=[-10,10]",
    N_stem=50,
    component_list=["radial"],
    is_2D_view=True,
    **dict_3D
)
#---------------------------------------------------------------

Many plots were made using the magnetic flux density, but they are of course applicable to any 2D output data, like the force computed using the airgap surface force:

In [ ]:
#---------------------------------------------------------------
# 2D fft of airgap surface force
out.force.AGSF.plot_3D_Data(
    "freqs->elec_order=[0,10]",
    "wavenumber->space_order=[-10,10]",
    N_stem=50,
    component_list=["radial"],
    is_2D_view=True,
    **dict_3D
)
#---------------------------------------------------------------